In [3]:
import numpy as np
from gc_utils import *
from testCases import *

In [4]:
def forward_propagation(x, theta):
    """
    implement J(theat) = x*theat
    :param x: 
    :param theat: 
    :return: 
    """
    
    J = theta * x
    
    return J

In [5]:
x, theta = 2, 4
J = forward_propagation(x, theta)
J

8

In [6]:
def backwrad_propagation(x, theta):
    """
    Compurte derivative of J with repect to theta 
    :param x: real number
    :param theta: real number
    :return: the gradient of the cost with respect to theta
    """
    
    dtheta = x
    
    return dtheta

In [8]:
x, theta = 2, 4
dtheta = backwrad_propagation(x, theta)
dtheta 

2